In [ ]:
import openai
import os
from pathlib import Path  
import json
import re
from dotenv import load_dotenv
env_path = Path('.') / 'secrets.env'
load_dotenv(dotenv_path=env_path)
openai.api_key =  os.environ.get("AZURE_OPENAI_API_KEY")
openai.api_base =  os.environ.get("AZURE_OPENAI_ENDPOINT")
openai.api_type = "azure"
openai.api_version = "2023-07-01-preview"
from tenacity import retry, wait_random_exponential, stop_after_attempt, stop_after_delay


In [ ]:
from datasets import load_dataset
import os
dataset = load_dataset("b-mc2/sql-create-context")
dataset_splits = {"train": dataset["train"]}

# os.makedirs("../data/", exist_ok=True)
# out_path="../data/sql-create-context.jsonl"
# for key, ds in dataset_splits.items():
#     with open(out_path, "w") as f:
#         for item in ds:
#             newitem = {
#                 "input": item["question"],
#                 "context": item["context"],
#                 "output": item["answer"],
#             }
#             f.write(json.dumps(newitem) + "\n")


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
all_data = pd.read_json("../data/sql-create-context.jsonl", lines=True)
# train_data, test_data = train_test_split(all_data, test_size=0.8)
all_data.head()
#extract table names from context and add to a new column table_names
all_data['table_names'] = all_data['context'].str.findall(r'(?<=\bCREATE TABLE\s)\w+')
#extract top 20 most frequent table names
top_20_tables = all_data['table_names'].explode().value_counts()[:20].index.tolist()
#filter data to only include rows with top 20 table names
all_data = all_data[all_data['table_names'].apply(lambda x: any([item in x for item in top_20_tables]))]
train_data, test_data = train_test_split(all_data, test_size=0.2)


In [ ]:
top_20_tables


In [ ]:

train_data.to_json("../llama2/data/sql-create-context-train.jsonl", orient="records", lines=True)
test_data.to_json("../llama2/data/sql-create-context-test.jsonl", orient="records", lines=True)

In [ ]:

list_data_dict = pd.read_json("../llama2/data/sql-create-context-train.jsonl", lines=True).to_dict(orient="records")
inputs = [item["input"] for item in list_data_dict]
outputs = [item["output"] for item in list_data_dict]

# dataset = Dataset.from_dict({"input": inputs, "output":outputs})   


### Data generation (Skip this step if data is already generated )

In [ ]:
schema = """
table: Categories, columns: CategoryID INTEGER, CategoryName TEXT, Description TEXT, Picture BLOB 
table: CustomerDemographics, columns: CustomerTypeID TEXT, CustomerDesc TEXT 
table: Customers, columns: CustomerID TEXT, CompanyName TEXT, ContactName TEXT, ContactTitle TEXT, Address TEXT, City TEXT, Region TEXT, PostalCode TEXT, Country TEXT, Phone TEXT, Fax TEXT 
table: Employees, columns: EmployeeID INTEGER, LastName TEXT, FirstName TEXT, Title TEXT, TitleOfCourtesy TEXT, BirthDate DATE, HireDate DATE, Address TEXT, City TEXT, Region TEXT, PostalCode TEXT, Country TEXT, HomePhone TEXT, Extension TEXT, Photo BLOB, Notes TEXT, ReportsTo INTEGER, PhotoPath TEXT 
table: EmployeeTerritories, columns: EmployeeID INTEGER, TerritoryID TEXT 
table: [Order Details], columns: OrderID INTEGER, ProductID INTEGER, UnitPrice NUMERIC, Quantity INTEGER, Discount REAL 
table: Orders, columns: OrderID INTEGER, CustomerID TEXT, EmployeeID INTEGER, OrderDate DATETIME, RequiredDate DATETIME, ShippedDate DATETIME, ShipVia INTEGER, Freight NUMERIC, ShipName TEXT, ShipAddress TEXT, ShipCity TEXT, ShipRegion TEXT, ShipPostalCode TEXT, ShipCountry TEXT 
table: Products, columns: ProductID INTEGER, ProductName TEXT, SupplierID INTEGER, CategoryID INTEGER, QuantityPerUnit TEXT, UnitPrice NUMERIC, UnitsInStock INTEGER, UnitsOnOrder INTEGER, ReorderLevel INTEGER, Discontinued TEXT 
table: Regions, columns: RegionID INTEGER, RegionDescription TEXT table: Shippers, columns: ShipperID INTEGER, CompanyName TEXT, Phone TEXT table: Suppliers, columns: SupplierID INTEGER, CompanyName TEXT, ContactName TEXT, ContactTitle TEXT, Address TEXT, City TEXT, Region TEXT, PostalCode TEXT, Country TEXT, Phone TEXT, Fax TEXT, HomePage TEXT table: Territories, columns: TerritoryID TEXT, TerritoryDescription TEXT, RegionID INTEGER
"""

In [ ]:
schema = schema.strip()
schema = schema.split("\n")
schema

In [ ]:
user_message = ""

# @retry(stop=(stop_after_delay(1) | stop_after_attempt(5)))
def generate_reading_comp_question(schema):
#     user_message =f""" 
#      You are training new employees to use the database. Given the following schema, generate at least 150 questions and answers that guide them to memorize the tables' schemas to write SQL query for business.
#      Try to give the business context to the question.
#         {schema}
#     You write the question and answer into multi-line json format as {{"question": "Some question", "answer": "Some answer"}}.
#     Your output:
# """
    user_message =f""" 
     You are training new employees to use the database. Given the following schema, generate at least 50 business questions and corresponding SQL query that can give answer to the question.
     <<Database schema>>
        {schema}
    Try to be creative in the business problem. Use join and aggregate functions to make the question more interesting.
    Output format: You write the question and sql query into multi-line json format as {{"question": "Some question", "sql_query": "Some answer"}}.
    Your output:
"""


    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a data analyst working with a database"},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']

# schema_comp = []
# for table in schema:
#     table_output = generate_reading_comp_question(table)
#     print(table_output)
#     schema_comp.append(table_output)
    
# schema_comp
outputs = []
for i in range(10):
    output = generate_reading_comp_question(schema)
    outputs.append(output)

In [ ]:

out_df = pd.read_json("../llama2/data/sql_examples.jsonl", lines=True)
out_df.to_json("../llama2/data/sql_examples.jsonl", orient="records", lines=True)

In [ ]:
result=""
#write code to remove all the numbered items like 1. 2. from the output
for output in outputs:
    out= re.sub(r'\d+\.\s+', '', outputs[1])
    result += out
print(result)